# 🚀 Enhanced Colab Processor with Improved Plot Capture
Captures ALL matplotlib plots including those saved with plt.savefig()

In [ ]:
# Setup
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client matplotlib -q

from google.colab import userdata
import json
import time
import io
import os
import base64
import tempfile
import traceback
from contextlib import redirect_stdout
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Matplotlib setup
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# Get service account from Colab secret (lowercase!)
sa_info = json.loads(userdata.get('sa_info'))
FOLDER_ID = sa_info.get('folder_id', '1S0gP-mWLQmnd060Atf8F2LpqEAZOdCjH')

# Create Drive service
creds = service_account.Credentials.from_service_account_info(
    sa_info,
    scopes=['https://www.googleapis.com/auth/drive']
)
drive_service = build('drive', 'v3', credentials=creds)

print('✅ Enhanced processor initialized')
print(f'📁 Monitoring folder: {FOLDER_ID}')

In [ ]:
# Enhanced plot capture system
class PlotCapture:
    """Captures all matplotlib plots during execution"""
    def __init__(self):
        self.captured_plots = []
        self.original_savefig = None
        self.original_show = None
        self.original_close = None
        
    def capture_figure(self, fig=None):
        """Capture a matplotlib figure as base64 PNG"""
        if fig is None:
            fig = plt.gcf()
            
        if fig.get_axes():  # Only capture if figure has content
            buf = io.BytesIO()
            fig.savefig(buf, format='png', bbox_inches='tight', dpi=150)
            buf.seek(0)
            self.captured_plots.append({
                'type': 'image/png',
                'data': base64.b64encode(buf.read()).decode('utf-8')
            })
            buf.close()
    
    def wrapped_savefig(self, *args, **kwargs):
        """Intercept savefig calls to capture plots"""
        # Capture the current figure before saving
        self.capture_figure(plt.gcf())
        # Call original savefig
        return self.original_savefig(*args, **kwargs)
    
    def wrapped_show(self, *args, **kwargs):
        """Intercept show calls to capture plots"""
        # Capture all current figures
        for fig_num in plt.get_fignums():
            self.capture_figure(plt.figure(fig_num))
        # Call original show
        return self.original_show(*args, **kwargs)
    
    def wrapped_close(self, fig=None):
        """Intercept close calls to capture plots before closing"""
        if fig is None:
            fig = plt.gcf()
        elif isinstance(fig, str) and fig == 'all':
            # Capture all figures before closing all
            for fig_num in plt.get_fignums():
                self.capture_figure(plt.figure(fig_num))
            return self.original_close(fig)
        
        # Capture specific figure before closing
        if hasattr(fig, 'number'):
            self.capture_figure(fig)
        
        return self.original_close(fig)
    
    def __enter__(self):
        # Store original functions
        self.original_savefig = plt.savefig
        self.original_show = plt.show
        self.original_close = plt.close
        
        # Replace with wrapped versions
        plt.savefig = self.wrapped_savefig
        plt.show = self.wrapped_show
        plt.close = self.wrapped_close
        
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        # Capture any remaining figures
        for fig_num in plt.get_fignums():
            self.capture_figure(plt.figure(fig_num))
        
        # Restore original functions
        plt.savefig = self.original_savefig
        plt.show = self.original_show
        plt.close = self.original_close
        
        # Close all figures to prevent memory leaks
        plt.close('all')

print('✅ Plot capture system ready')

In [ ]:
# Main processor loop with enhanced plot capture
print('🚀 Starting enhanced processor...')
processed_commands = set()

def execute_with_plots(code):
    """Execute code and capture output + plots"""
    stdout_buffer = io.StringIO()
    error = None
    plots = []
    
    # Execute with plot capture
    try:
        with redirect_stdout(stdout_buffer):
            with PlotCapture() as capture:
                # Create a proper execution environment
                exec_globals = {
                    '__name__': '__main__',
                    '__builtins__': __builtins__,
                    'plt': plt,
                    'matplotlib': matplotlib
                }
                
                # Execute the code
                exec(code, exec_globals)
                
                # Get captured plots
                plots = capture.captured_plots
                
    except Exception as e:
        error = traceback.format_exc()
    
    return {
        'status': 'error' if error else 'success',
        'output': stdout_buffer.getvalue(),
        'error': error,
        'visualizations': plots,
        'output_type': 'rich' if plots else 'text',
        'processor': 'enhanced_processor'
    }

while True:
    try:
        # Find command files
        query = f"'{FOLDER_ID}' in parents and name contains 'command_' and trashed=false"
        results = drive_service.files().list(q=query, fields='files(id, name)').execute()
        
        for file in results.get('files', []):
            # Read command
            content = drive_service.files().get_media(fileId=file['id']).execute()
            command = json.loads(content.decode('utf-8'))
            
            if command['id'] in processed_commands:
                continue
                
            print(f"📥 Processing: {command['id']}")
            
            # Delete command file
            try:
                drive_service.files().delete(fileId=file['id']).execute()
            except:
                pass
            
            # Execute with plot capture
            result = execute_with_plots(command.get('code', ''))
            result['command_id'] = command['id']
            
            # Write result with correct naming pattern (double underscore)
            result_name = f"result_result_{command['id'].replace('cmd_', '')}.json"
            
            with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
                json.dump(result, f, indent=2)
                temp_path = f.name
            
            try:
                media = MediaFileUpload(temp_path, mimetype='application/json')
                drive_service.files().create(
                    body={
                        'name': result_name,
                        'parents': [FOLDER_ID]
                    },
                    media_body=media
                ).execute()
            finally:
                os.unlink(temp_path)
            
            processed_commands.add(command['id'])
            
            if result.get('visualizations'):
                print(f"✅ Done! Captured {len(result['visualizations'])} plots")
            else:
                print(f"✅ Done!")
            
        time.sleep(1)
        
    except KeyboardInterrupt:
        print("\n👋 Stopped")
        break
    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()
        time.sleep(5)